# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [1]:
import pandas as pd

def cargar_datos():
    colspecs = [(0, 10), (10, 40), (40, 50), (50, 55)]
    column_names = ["fecha", "producto", "precio", "cantidad"]
    
    df = pd.read_fwf('datos.dat', colspecs=colspecs, names=column_names)
    return df

datos = cargar_datos()
print(datos)


         fecha    producto  precio  cantidad
0   2024-08-27     Mirinda    1510        14
1   2024-08-27     Mirinda    1560        12
2   2024-08-28     Torasso     940         8
3   2024-08-29  Pepsi Cola    1210        10
4   2024-08-30     Mirinda    1520         1
5   2024-09-01     Mirinda    1550        15
6   2024-09-01      Sprite     810         4
7   2024-09-02   Coca Cola    1100         4
8   2024-09-02  Pepsi Cola    1220        13
9   2024-09-02     Torasso     910         5
10  2024-09-02     Torasso     920         3
11  2024-09-03   Coca Cola    1020         8
12  2024-09-03     Mirinda    1570         7
13  2024-09-03     Mirinda    1590         2
14  2024-09-04  Pepsi Cola    1220        13
15  2024-09-05     Mirinda    1500         3
16  2024-09-05  Pepsi Cola    1300         5
17  2024-09-06   Coca Cola    1080         1
18  2024-09-06      Sprite     860        12
19  2024-09-06     Torasso     930         3
20  2024-09-07   Coca Cola    1080        14
21  2024-0

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [3]:
def calcular_totales(datos):
    datos['precio'] = pd.to_numeric(datos['precio'], errors='coerce')
    datos['cantidad'] = pd.to_numeric(datos['cantidad'], errors='coerce')
    
    importe_total = (datos['precio'] * datos['cantidad']).sum()
    cantidad_total = datos['cantidad'].sum()
    
    return importe_total, cantidad_total

importe, cantidad = calcular_totales(datos)

print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")

Las ventas fueron de $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [5]:
def unidades_vendidas(datos):
    datos['cantidad'] = pd.to_numeric(datos['cantidad'], errors='coerce')
    unidades_totales = datos.groupby('producto')['cantidad'].sum()
    for producto, cantidad in unidades_totales.items():
        print(f"{producto}: {cantidad} unidades")

unidades_vendidas(datos)

Coca Cola: 57 unidades
Mirinda: 85 unidades
Pepsi Cola: 89 unidades
Sprite: 72 unidades
Torasso: 32 unidades


###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [6]:
def precio_promedio(datos):
    datos['precio'] = pd.to_numeric(datos['precio'], errors='coerce')
    
    precio_promedio_productos = datos.groupby('producto')['precio'].mean()
    for producto, precio in precio_promedio_productos.items():
        print(f"{producto}: Precio promedio ${precio:.2f}")

precio_promedio(datos)

Coca Cola: Precio promedio $1072.50
Mirinda: Precio promedio $1545.83
Pepsi Cola: Precio promedio $1245.00
Sprite: Precio promedio $841.43
Torasso: Precio promedio $920.00


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [7]:
def ranking_productos(datos, top=3):
    datos['cantidad'] = pd.to_numeric(datos['cantidad'], errors='coerce')
    
    unidades_totales = datos.groupby('producto')['cantidad'].sum()
    
    ranking = unidades_totales.sort_values(ascending=False).head(top)
    for producto, cantidad in ranking.items():
        print(f"{producto}: {cantidad} unidades vendidas")

ranking_productos(datos)

Pepsi Cola: 89 unidades vendidas
Mirinda: 85 unidades vendidas
Sprite: 72 unidades vendidas


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [8]:
def ventas_por_mes(datos):
    datos['fecha'] = pd.to_datetime(datos['fecha'], errors='coerce')
    datos['cantidad'] = pd.to_numeric(datos['cantidad'], errors='coerce')
    
    datos['mes_anio'] = datos['fecha'].dt.to_period('M')
    
    ventas_mensuales = datos.groupby(['mes_anio', 'producto'])['cantidad'].sum().reset_index()
    
    for _, row in ventas_mensuales.iterrows():
        print(f"{row['mes_anio']}: {row['producto']} - {row['cantidad']} unidades vendidas")

ventas_por_mes(datos)

2024-08: Mirinda - 27 unidades vendidas
2024-08: Pepsi Cola - 10 unidades vendidas
2024-08: Torasso - 8 unidades vendidas
2024-09: Coca Cola - 57 unidades vendidas
2024-09: Mirinda - 58 unidades vendidas
2024-09: Pepsi Cola - 79 unidades vendidas
2024-09: Sprite - 72 unidades vendidas
2024-09: Torasso - 24 unidades vendidas


### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [9]:
def resumen_ventas(datos):
    datos['precio'] = pd.to_numeric(datos['precio'], errors='coerce')
    datos['cantidad'] = pd.to_numeric(datos['cantidad'], errors='coerce')
    
    resumen = datos.groupby('producto').agg(
        precio_promedio=('precio', 'mean'),
        unidades_vendidas=('cantidad', 'sum'),
        importe_total=('precio', lambda x: (x * datos.loc[x.index, 'cantidad']).sum())
    ).reset_index()
    
    resumen = resumen.sort_values(by='producto')
    
    for _, row in resumen.iterrows():
        print(f"{row['producto']}: Precio promedio ${row['precio_promedio']:.2f}, {row['unidades_vendidas']} unidades vendidas, Importe total ${row['importe_total']:.2f}")


resumen_ventas(datos)

Coca Cola: Precio promedio $1072.50, 57 unidades vendidas, Importe total $60780.00
Mirinda: Precio promedio $1545.83, 85 unidades vendidas, Importe total $131080.00
Pepsi Cola: Precio promedio $1245.00, 89 unidades vendidas, Importe total $110510.00
Sprite: Precio promedio $841.43, 72 unidades vendidas, Importe total $61040.00
Torasso: Precio promedio $920.00, 32 unidades vendidas, Importe total $29320.00


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [45]:
import numpy as np

def f(x, coeficientes):
    a,b,c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return y - y_pred

X = np.array([0,1,2,3,5])
Y = np.array([0,8,12,12,0])

def buscar_coeficientes():
    coeficientes = np.random.uniform(-10, 10, 3)
    aprendizaje = 0.01
    tolerancia = 1e-2

    for _ in range(10000): 
        Y_pred = f(X, coeficientes)
        error = Y - Y_pred
        if np.all(np.abs(error) < tolerancia):
            return coeficientes

        coeficientes += aprendizaje * np.dot(error, np.vstack([X**2, X, np.ones(len(X))]).T) / len(X)
    
    return None


coeficientes = buscar_coeficientes()
coeficientes

array([-1.9984148 ,  9.99046646,  0.00999741])

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [56]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a,b,c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return np.sum((y - y_pred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8,12,11, 1]) # Observar que no son los mismos valores que en el ejemplo anterior

def buscar_coeficientes():
    coeficientes = randint(-10, 10, 3)
    
    Y_pred = f(X, coeficientes)
    error_actual = error(Y, Y_pred)
    
    aprendizaje = 0.001
    while error_actual > 1:
        nuevos_coeficientes = coeficientes + aprendizaje * randint(-10, 10, 3)
        
        Y_pred_nuevo = f(X, nuevos_coeficientes)
        error_nuevo = error(Y, Y_pred_nuevo)
        
        if error_nuevo < error_actual:
            coeficientes = nuevos_coeficientes
            error_actual = error_nuevo
    
    return coeficientes

coeficientes = buscar_coeficientes()
coeficientes

array([-1.777,  8.957,  0.616])